In [86]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

In [87]:
bike_master = pd.read_csv('bike_full.csv', dtype={5: str, 7: str})  # Convert both columns to strings

In [88]:
# before_cleaning = bike_master.memory_usage(deep=True).sum() / 1024 ** 2
# print(f"Memory usage before cleaning: {before_cleaning:.2f} MB")

In [89]:
bike_master.isnull().sum()

ride_id                    0
rideable_type              0
started_at                 0
ended_at                   0
start_station_name    742776
start_station_id      742776
end_station_name      811282
end_station_id        811282
start_lat                  2
start_lng                  2
end_lat                16765
end_lng                16765
member_casual              0
dtype: int64

In [90]:
bike_master_clean = bike_master.copy()

# Convert datetime columns
bike_master_clean['started_at'] = pd.to_datetime(bike_master_clean['started_at'])
bike_master_clean['ended_at'] = pd.to_datetime(bike_master_clean['ended_at'])

# Convert category columns
category_columns = ['rideable_type', 'start_station_name', 'end_station_name', 'member_casual']
for col in category_columns:
    bike_master_clean[col] = bike_master_clean[col].astype('category')

# Convert string columns (IDs)
string_columns = ['ride_id', 'start_station_id', 'end_station_id']
for col in string_columns:
    bike_master_clean[col] = bike_master_clean[col].astype('string')



In [91]:
#dropping ride_id column
bike_master_clean = bike_master_clean.drop(columns=['ride_id'])

In [92]:
after_cleaning = bike_master_clean.memory_usage(deep=True).sum() / 1024 ** 2
print(f"Memory usage after cleaning: {after_cleaning:.2f} MB")

Memory usage after cleaning: 1693.02 MB


In [93]:
bike_master_clean.head()

,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,classic_bike,2021-01-01 00:08:33,2021-01-01 00:33:53,Maine Ave & 9th St SW,31646.0,Rosslyn Metro / Wilson Blvd & Ft Myer Dr,31015.0,38.880440,-77.025236,38.894600,-77.072305,member
1,electric_bike,2021-01-01 00:13:43,2021-01-01 00:29:34,10th & U St NW,31111.0,NaN,<NA>,38.917193,-77.025894,38.960000,-77.020000,casual
2,electric_bike,2021-01-01 00:14:32,2021-01-01 00:28:45,17th & Corcoran St NW,31214.0,14th & Belmont St NW,31119.0,38.912138,-77.038568,38.920870,-77.031691,member
3,classic_bike,2021-01-01 00:15:45,2021-01-01 00:21:20,Wilson Blvd. & N. Vermont St.,31926.0,Wilson Blvd. & N. Vermont St.,31926.0,38.879477,-77.114563,38.879477,-77.114563,member
4,classic_bike,2021-01-01 00:17:46,2021-01-01 00:21:00,11th & Park Rd NW,31651.0,14th & Newton St NW,31649.0,38.931322,-77.028247,38.931991,-77.032956,member


In [115]:
bike_master_clean.isnull().sum()

rideable_type              0
started_at                 0
ended_at                   0
start_station_name    742776
start_station_id      742776
end_station_name      811281
end_station_id        811281
start_lat                  2
start_lng                  2
end_lat                16765
end_lng                16765
member_casual              0
dtype: int64

In [127]:
print(bike_master_clean[['start_station_name', 'end_station_name', 'start_station_id', 'end_station_id', 'end_lat', 'end_lng']].drop_duplicates())

                                         start_station_name  \
0                                     Maine Ave & 9th St SW   
1                                            10th & U St NW   
2                                     17th & Corcoran St NW   
3                             Wilson Blvd. & N. Vermont St.   
4                                         11th & Park Rd NW   
...                                                     ...   
10693245  Falls Church City Hall / Park Ave & Little Fal...   
10693345                       Massachusetts & Idaho Ave NW   
10693464                         Langston Blvd & N Adams St   
10693623       Pentagon Row Plaza / Army Navy Dr & Joyce St   
10693783                          MLK & Marion Barry Ave SE   

                                  end_station_name start_station_id  \
0         Rosslyn Metro / Wilson Blvd & Ft Myer Dr          31646.0   
1                                              NaN          31111.0   
2                             